# Centerpartiets budgetmotion 2022

https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/centerpartiets-budgetmotion-2022_H9024121

In [1]:
import pandas as pd
import requests

pd.options.mode.chained_assignment = None

In [2]:
multiplier = 1_000_000

In [35]:
docs = [
    {'utgiftsområde': 1, 'dok_id': 'H9024141'},
    {'utgiftsområde': 2, 'dok_id': 'H9024140'},
    {'utgiftsområde': 3, 'dok_id': 'H9024142'},
    {'utgiftsområde': 4, 'dok_id': 'H9024143'},
    {'utgiftsområde': 5, 'dok_id': 'H9024144'},
    {'utgiftsområde': 6, 'dok_id': 'H9024145'},
    {'utgiftsområde': 7, 'dok_id': 'H9024146'},
    {'utgiftsområde': 8, 'dok_id': 'H9024147'},
    {'utgiftsområde': 9, 'dok_id': 'H9024128'},
    {'utgiftsområde': 10, 'dok_id': 'H9024148'},
    {'utgiftsområde': 11, 'dok_id': 'H9024149'},
    {'utgiftsområde': 12, 'dok_id': 'H9024150'},
    {'utgiftsområde': 13, 'dok_id': 'H9024127'},
    {'utgiftsområde': 14, 'dok_id': 'H9024129'},
    {'utgiftsområde': 15, 'dok_id': 'H9024125'},
    {'utgiftsområde': 16, 'dok_id': 'H9024126'},
    {'utgiftsområde': 17, 'dok_id': 'H9024130'},
    {'utgiftsområde': 18, 'dok_id': 'H9024122'},
    {'utgiftsområde': 19, 'dok_id': 'H9024123'},
    {'utgiftsområde': 20, 'dok_id': 'H9024124'},
    {'utgiftsområde': 21, 'dok_id': 'H9024136'},
    {'utgiftsområde': 22, 'dok_id': 'H9024135'},
    {'utgiftsområde': 23, 'dok_id': 'H9024134'},
    {'utgiftsområde': 24, 'dok_id': 'H9024133'},
    {'utgiftsområde': 25, 'dok_id': 'H9024132'}]

In [36]:
def find_matching_table(tables, loc=None):
    if loc:
        return tables[loc]
    for table in tables:
        if table.columns.shape == (5,):
            break
    return table

In [37]:
def fetch_table(url, area):
    tables = pd.read_html(url, encoding='utf8', header=2)
    cols = ['Anslag', 'Namn', '2022', '2023', '2024']
    loc = 3 if area in [14, 24] else None
    df = find_matching_table(tables, loc)
    df.columns = cols
    df = df.dropna(how='all')
    df = df[~df.Anslag.str.startswith('Summa', na=False)]
    df['Utgiftsområde'] = area
    return df

In [43]:
tables = []

for doc in docs:
    url = f'http://data.riksdagen.se/dokument/{doc["dok_id"]}.html'
    table = fetch_table(url, area=doc['utgiftsområde'])
    tables.append(table)

In [44]:
df = pd.concat(tables, sort=False)

In [45]:
df = df.dropna(how='all')

In [47]:
for col in ['2022', '2023', '2024']:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.split('.', expand=True)[0]
    df[col] = df[col].str.replace('±0', '0', regex=False)
    df[col] = df[col].str.replace('\s+', '', regex=True)
    df[col] = df[col].str.replace('−', '-')
    df[col] = df[col].astype(int) * multiplier

In [49]:
df.to_csv('../data/budgetmotion-2022-c.csv', index=False)